# 動的計画法でCMDPを解く

参考：
* [Approximate Constrained Discounted Dynamic Programming with Uniform Feasibility and Optimality](https://arxiv.org/abs/2308.03297)
* [Dynamic programming equations for discounted constrained stochastic control](https://ieeexplore.ieee.org/abstract/document/1298995)：基本の論文だけど履歴依存＆non-stationary
* [Non-randomized control of constrained Markov decision processes](https://ieeexplore.ieee.org/document/1656446)：基本の論文だけどnon-stationary

基本的にはCMDPはラグランジュ形式や線形計画法で解きます．
しかし，ラグランジュ形式は不安定な問題があったり，線形計画法は高次元な問題に使えにくかったり，それぞれに問題があります．
一方で，CMDPを動的計画法で解こうとする試みはいくつかあります．見てみましょう．

表記：
* 方策 $\pi$：状態$X$から行動$A$へのマップ．決定的な行動と同じ．
* 遷移確率：$P_{x y}^{\pi(x)}$
* 報酬，コスト：$R(x, \pi(x))$，$C(x, \pi(x))$
* 累積報酬，累積コスト：

$$
\begin{aligned}
& V^\pi(x)=E\left[\sum_{t=0}^{\infty} \gamma^t R\left(X_t, \pi\left(X_t\right)\right) \mid X_0=x\right] \\
& J^\pi(x)=E\left[\sum_{t=0}^{\infty} \beta^t C\left(X_t, \pi\left(X_t\right)\right) \mid X_0=x\right]
\end{aligned}
$$

* 方策 → 方策の集合 へのマップ（$\pi$についてuniformly-feasibleな集合）：$\Phi(\pi)=\left\{g \mid g \in \Pi, J^g \leq J^\pi\right\}, \pi \in \Pi$
    * $g \in \Phi(\pi)$のとき，$g$は$\pi$-feasibleと呼ぶ．

今回の特別な表記ですが，$x \in X, \pi \in \Pi$について，最適価値関数を次のように定義します：

$$
V_c^*(x, \pi)=\max _{g \in \Phi(\pi)} V^g(x)
$$

これは何らかの方策$\pi$について定義されてることに注意しましょう！普通は状態についてだけ定義されますが，CMDPの関係上，$\pi$についても引数を取ります．

---

**問題設定の目標**

MDPを$M_{\pi^c}^c\left(x_0\right)=\left(X, A, P, R, C, \pi^c, x_0\right)$として定義します．
ここで，$\pi^c\in \Pi$はthreshold用の方策とします．
今回の目標は，

$$
V_c^*\left(x_0, \pi^c\right)=V^{\pi_c^*}\left(x_0\right)
$$

であるような，$\pi^*_c \in \Phi(\pi^c)$を見つけることとします．
つまり，$\pi^*_c$は$\pi^c$と同じもしくはそれ以下のコストを実現する方策のうち，最大のリターンを達成する方策ですね．

これは$x_0$が固定であり，かつ$\Phi(\pi^c)$が常に空ではないので，$\pi^*_c$は必ず存在します．

---

普通のCMDPでは次の問題を解くことに注意しましょう．ここで$\delta$は初期状態分布です：

$$
\underset{\pi \in \Phi_\delta\left(\pi^c\right)}{\arg \max }\left(\sum_{x \in X} \delta(x) V^\pi(x)\right)
$$

$$
\Phi_\delta\left(\pi^c\right)=\left\{g \mid g \in \Pi, \sum_{x \in X} \delta(x) J^g(x) \leq \sum_{x \in X} \delta(x) J^{\pi^c}(x)\right\}
$$

しかし，これは実はNP困難になります（non-randomizedのときだけ？要検証．[Constrained Discounted Dynamic Programming](https://www.researchgate.net/publication/2656018_Constrained_Discounted_Dynamic_Programming)参照）．

以下，$M_{\pi^c}^c\left(x_0\right)=\left(X, A, P, R, C, \pi^c, x_0\right)$を解くことを考えましょう．


## $M_{\pi^c}^c\left(x_0\right)=\left(X, A, P, R, C, \pi^c, x_0\right)$を解くことを考えましょう

$\pi\in \Pi$について，$T_\pi: B(X) \rightarrow B(X)$ および $U_\pi: B(X) \rightarrow B(X)$を次のように定義します：

* 報酬についてのベルマン作用素：$T_\pi(u)(x)=R(x, \pi(x))+\gamma \sum_{y \in X} P_{x y}^{\pi(x)} u(y)$
* コストについてのベルマン作用素：$U_\pi(u)(x)=C(x, \pi(x))+\beta \sum_{y \in X} P_{x y}^{\pi(x)} u(y)$

また，
* $A^\pi$を空でない$A$の部分集合とし，任意の$x\in X$について，$\pi(x)\in A^\pi(x)$および$A^\pi(x) \subseteq A(x)$とする．つまり，$x$で取れる行動の集合のうち，$\pi(x)$が取る行動を含む部分集合です．
* $F: \Pi \to 2^\Pi$：$\pi \in \Pi$について，$F(\pi)=\left\{g \mid g \in \Pi, g(x) \in A^\pi(x) \forall x \in X\right\}$とする．つまり，方策の出力が$A^\pi$に含まれるような方策の集合です．
    * $F(\pi)$が$\Phi(\pi)$の部分集合の時，つまり$F(\pi)$に含まれる方策が$\pi$を使って表現された制約を満たす時，$F(\pi)$は「$A^\pi$によって誘導された$\pi$-feasibleな方策の集合」と呼びます．
    * もし全ての$\pi\in \Pi$について，$A^\pi$が$\pi$-feasibleな方策の集合を誘導する場合，$\left\{A^\pi, \pi \in \Pi\right\}$を「DPに誘導可能な集合」と呼びます．

例えば$x\in X$と$\pi \in \Pi$について，

$$
A^\pi(x)=\left\{a \mid a \in A(x), C(x, a)+\beta \sum_{y \in X} P_{x y}^a J^\pi(y) \leq J^\pi(x)\right\}
$$

とすれば，任意の$\phi \in F(\pi)$について$U_\phi(J^\pi) \leq J^\pi$なので，$J^\phi \leq J^\pi$です．
つまりDP-inducibleな$\{A^\pi, \pi\in \Pi\}$では，$x\in X$と$\pi$について，「$x$からその後$\pi$に従って動いたときに，制約が満たされるような最初の行動の集合」が与えられます．

以下，このようなDP-inducibleな集合$\left\{A^\pi, \pi \in \Pi\right\}$がアクセス可能であると仮定して話を進めます．（つまり，二ステップ目以降に各$\pi$について動くとした時，最初のステップで取って良い行動のリストが分かっている状況）

次の一様最適な価値関数を定義します：

$$
V^*(x, \pi)=\max _{g \in F(\pi)} V^g(x)
$$

$F(\pi)$の中身は制約を満たすので，これは制約を満たす中でReturnを最大にする方策を見つけるのと他なりません．
実際，次の定理が成立します．

---

任意の$\pi \in \Pi$について，$V^*(x, \pi)=V^{\pi^*}(x)$が任意の$x\in X$について成立するような，一様に最適な$\pi$-feasibleな$\pi^*\in F(\pi)$が存在する．

**証明**

適当な$\pi \in \Pi$について，$\left(X, A^\pi, P, R\right)$なる非制約付きMDPを考えてみましょう．
$F(\pi)=\left\{g \mid g \in \Pi, g(x) \in A^\pi(x) \forall x \in X\right\}$は方策の出力が$A^\pi$に含まれる方策の集合なので，このMDPの最適方策は当然$F(\pi)$に入っています．

ここで，$u \in B(X \times \Pi)$について，次の作用素$T_F: B(X \times \Pi) \rightarrow B(X \times \Pi)$を定義します：

$$
T_F(u)(x, \pi)=\max _{g \in F(\pi)}\left(R(x, g(x))+\gamma \sum_{y \in X} P_{x y}^{g(x)} u(y, g)\right)
$$

まず，これは$\gamma$-contractionです．
実際，

$$
\begin{aligned}
& \left\|T_F(u)-T_F(v)\right\|_{\infty} \\
& \quad \leq \max _{x \in X} \max _{\pi \in \Pi} \max _{g \in F(\pi)}\left(\gamma \sum_{y \in X} P_{x y}^{g(x)}|u(y, g)-v(y, g)|\right) \\
& \quad \leq \gamma\|u-v\|_{\infty}
\end{aligned}
$$

です．よって，バナッハの不動点定理から，$T_F(u)=u$となる不動点が存在します．
実はこの不動点は$T_F\left(V^*\right)=V^*$を満たします．

まず，

$$
\begin{aligned}
& V^*(x, \pi)=\max _{g \in F(\pi)} V^g(x) \text { by the definition of } V^* \\
& \quad=\max _{g \in F(\pi)}\left(R(x, g(x))+\gamma \sum_{y \in X} P_{x y}^{g(x)} V^g(y)\right) \text { by the definition of } V^g \\
& \quad \leq \max _{g \in F(\pi)}\left(R(x, g(x))+\gamma \sum_{y \in X} P_{x y}^{g(x)} \max _{\sigma \in F(g)} V^\sigma(y)\right) \text { because } g \in F(g) \\
& \quad=\max _{g \in F(\pi)}\left(R(x, g(x))+\gamma \sum_{y \in X} P_{x y}^{g(x)} V^*(y, g)\right) \text { by the definition of } V^* \\
& \quad=T_F\left(V^*\right)(x, \pi)
\end{aligned}
$$

です．
一方で，$\pi^* \in F(\pi)$ such that $V^*(y, \pi)=V^{\pi^*}(y)$となる$\pi^*$が存在するので，

$$
\begin{aligned}
V^* & (x, \pi)=V^{\pi^*}(x) \\
& =R\left(x, \pi^*(x)\right)+\gamma \sum_{y \in X} P_{x y}^{\pi^*(x)} V^{\pi^*}(y) \\
& =\max _{g \in F(\pi)}\left(R(x, g(x))+\gamma \sum_{y \in X} P_{x y}^{g(x)} V^{\pi^*}(y)\right) \text { by } T_F\left(V^*\right)=V^* \text { and the property of } \pi^* \\
& \geq R(x, \sigma(x))+\gamma \sum_{y \in X} P_{x y}^{\sigma(x)} V^{\pi^*}(y) \text { for any } \sigma \in F(\pi) \\
& =R(x, \sigma(x))+\gamma \sum_{y \in X} P_{x y}^{\sigma(x)} V^*(y, \pi) \text { by the property of } \pi^* \\
& \geq R(x, \sigma(x))+\gamma \sum_{y \in X} P_{x y}^{\sigma(x)} V^*(y, \sigma) \text { because } \sigma \in F(\pi)
\end{aligned}
$$

が成立します．よって不動点は$V^*$です．


---
